In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import glob
from PIL import Image, ImageFilter, ImageOps
from scipy.stats import kurtosis, skew

In [2]:
tumors = [cv2.imread(file) for file in glob.glob("C:/Users/devda/Major/Data/all-mias/JPEG/Tumors/*.jpg")]
for i in range(119):
    tumors[i] = cv2.cvtColor(tumors[i], cv2.COLOR_BGR2GRAY)

In [3]:
df = pd.read_csv('C:/Users/devda/Major/Data/Info.csv')
df.X = df.X.fillna(1024)
df.RADIUS = df.RADIUS.fillna(0)
df.X = df.X.astype(int)
df.RADIUS = df.RADIUS.astype(int)

In [4]:
tdf = df[df['RADIUS'] > 0] 
tdf.reset_index(inplace = True)

In [5]:
roundness = [0] * tdf.shape[0]
acreage_ratio = [0] * tdf.shape[0]
energy = [0] * tdf.shape[0]
entropy = [0] * tdf.shape[0]
inv_moment = [0] * tdf.shape[0]
contrast_coeff = [0] * tdf.shape[0]
tmean = [0] * tdf.shape[0]
tvar = [0] * tdf.shape[0]
hmean = [0] * tdf.shape[0]
hvar = [0] * tdf.shape[0]
hpeak = [0] * tdf.shape[0]
hskew = [0] * tdf.shape[0]
#idx = [0] * tdf.shape[0]

for i in range(119):
    (x, y) = (0, 0)
    r = 0
    a = tumors[i].max()
    _, th = cv2.threshold(tumors[i], int(a/2)+111, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    maxi = 0
    #idx = 0
    for j in range(1, len(contours)):
        if cv2.contourArea(contours[j]) > maxi:
            maxi = cv2.contourArea(contours[j])
            idx = j
    #print(len(contours), idx[i])
    if idx<len(contours):
        (x,y), r = cv2.minEnclosingCircle(contours[idx])
        perimeter = cv2.arcLength(contours[idx], True) 
        area = 4*3.14*3.14*r**2
        roundness[i] = (perimeter**2)/(4*3.14*area)
        #print(perimeter, area, roundness[i])
        acreage_ratio[i] = cv2.contourArea(contours[idx])/(3.14*r**2)
    new_img = np.zeros_like(th)
    for val in np.unique(th)[1:]:                                      
        mask = np.uint8(th == val)                                     
        labels, stats = cv2.connectedComponentsWithStats(mask, 4)[1:3]  
        largest_label = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])      
        new_img[labels == largest_label] = val    
    new_img = cv2.bitwise_and(tumors[i], tumors[i], mask=new_img)
    new_new_img = np.zeros_like(new_img)
    new_new_img = new_img/255
    tmean[i] = np.mean(new_new_img)
    tvar[i] = np.var(new_new_img)
    for h in range(tumors[i].shape[0]):
        for k in range(tumors[i].shape[1]):
            energy[i] = energy[i] + new_new_img[h, k]**2
            if new_new_img[h, k]!=0:
                entropy[i] = entropy[i] + new_new_img[h, k] * (-np.log(new_new_img[h, k]))
            inv_moment[i] = inv_moment[i] + new_new_img[h, k] / (1 + (h-k)**2)
            contrast_coeff[i] = contrast_coeff[i] + ((h-k)**2)*new_new_img[h, k]
    #print(h, k, tumors[i].shape[0], tumors[i].shape[1])
    energy[i] = energy[i]/ (tumors[i].shape[0]*tumors[i].shape[1])
    hist = cv2.calcHist([tumors[i]], [0], None, [256], [0,256])
    hpeak[i] = max(hist)[0]
    hmean[i] = np.mean(hist)
    hvar[i] = np.var(hist)
    hskew[i] = skew(hist)[0]

In [7]:
tdf['ROUNDNESS'] = roundness
tdf['ACREAGE_RATIO'] = acreage_ratio
tdf['ENERGY'] = energy
tdf['ENTROPY'] = entropy
tdf['INVERSE_MOMENT'] = inv_moment
tdf['CONTRAST_COEFFICIENT'] = contrast_coeff
tdf['TEXTURE_MEAN'] = tmean
tdf['TEXTURE_VARIANCE'] = tvar
tdf['HISTOGRAM_MEAN'] = hmean
tdf['HISTOGRAM_VARIANCE'] = hvar
tdf['HISTOGRAM_PEAK'] = hpeak
tdf['HISTOGRAM_SKEW'] = hskew

C:\Users\devda\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\devda\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\devda\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [8]:
tdf.head()

,index,REFNUM,BG,CLASS,SEVERITY,X,Y,RADIUS,ROUNDNESS,ACREAGE_RATIO,ENERGY,ENTROPY,INVERSE_MOMENT,CONTRAST_COEFFICIENT,TEXTURE_MEAN,TEXTURE_VARIANCE,HISTOGRAM_MEAN,HISTOGRAM_VARIANCE,HISTOGRAM_PEAK,HISTOGRAM_SKEW
0,0,0,G,CIRC,B,535,599,197,0.179829,0.464240,0.124454,604.433566,230.976373,1.274185e+08,0.128265,0.108002,606.390625,4.293282e+06,33244.0,15.265376
1,1,1,G,CIRC,B,522,744,69,0.179708,0.500457,0.170766,118.418474,82.284489,6.472679e+06,0.176849,0.139491,74.390625,5.335095e+04,3336.0,11.236760
2,4,4,F,CIRC,B,477,891,30,0.059999,0.563405,0.004066,1.678343,0.100019,2.878486e+03,0.004509,0.004046,14.062500,1.678191e+03,502.0,7.668410
3,5,4,F,CIRC,B,500,856,26,0.052569,0.355713,0.010535,3.748092,0.133102,9.320639e+03,0.011843,0.010395,10.562500,1.737707e+03,374.0,5.719888
4,10,9,F,CIRC,B,525,599,33,0.143369,0.388731,0.135623,52.552700,63.185099,8.988756e+04,0.147203,0.113954,17.015625,3.064242e+03,657.0,7.220394


In [ ]:
tdf.to_csv('Features.csv') 